## **Sabrina Zheng | Group 6 | Project 1**

_Written in collaboration with ChatGPT from OpenAI to improve understanding, assist with the explanation of the query, and enhance formatting and display of the queries._

## **Top 3 Queries:**

### **1.**

**Proposition:** <span style="font-size:14px;">Write a query to find the Pokemon that has the highest attack in Type1 and Type2 within each generation</span>

**Tables:**

- PokemonGen1.dbo.PokemonGen1
- PokemonGen2.dbo.PokemonGen2
- PokemonGen3.dbo.PokemonGen3

**Columns:**

- ID (or PokemonNoGen1, PokemonNoGen2, PokemonNoGen3)
- Name
- Attack
- Type1 and Type2
- Generation

**Predicate:**

- The UnifiedPokemon CTE is used to create a unified view of the Pokemon from all 3 generations by getting the IDs, names, attack values, types, and assigns a generation tag based on the source table
- Then the TypeRankings CTE creates a temporary dataset that consolidates the Pokemon types and then ranks the Pokemon by their attack values within each type and generation using the RANK() function
- The query then selects the TypeRankings CTE and filters for when the Pokemon with the highest attack within the type and generation

In [12]:
-- Create a unified dataset of Pokemon from all generations
WITH UnifiedPokemon AS (
    SELECT PokemonNoGen1 AS ID, Name, Attack, Type1, Type2, 'Gen1' AS Generation FROM PokemonGen1.dbo.PokemonGen1
    UNION ALL
    SELECT PokemonNoGen2 AS ID, Name, Attack, Type1, Type2, 'Gen2' FROM PokemonGen2.dbo.PokemonGen2
    UNION ALL
    SELECT PokemonNoGen3 AS ID, Name, Attack, Type1, Type2, 'Gen3' FROM PokemonGen3.dbo.PokemonGen3
),

-- Rank Pokemon within each type and generation based on their attack stat
TypeRankings AS (
    SELECT
        Name,
        Attack,
        Type,
        Generation,
        RANK() OVER(PARTITION BY Type, Generation ORDER BY Attack DESC) AS RankInType
    FROM (
        SELECT Name, Attack, Type1 AS Type, Generation FROM UnifiedPokemon
        UNION ALL
        SELECT Name, Attack, Type2, Generation FROM UnifiedPokemon WHERE Type2 IS NOT NULL
    ) AS AllTypes
)

-- Select Pokemon with the highest attack stat within each type and generation
SELECT Type, Generation, Name, Attack
FROM TypeRankings
WHERE RankInType = 1
ORDER BY Type, Generation;


(62 rows affected)

Total execution time: 00:00:00.008

Type,Generation,Name,Attack
Bug,Gen1,Pinsir,125
Bug,Gen2,Scizor,130
Bug,Gen3,Armaldo,125
Dark,Gen2,Tyranitar,134
Dark,Gen3,Absol,130
Dragon,Gen1,Dragonite,134
Dragon,Gen2,Kingdra,95
Dragon,Gen3,Rayquaza,150
Electric,Gen1,Zapdos,90
Electric,Gen1,Raichu,90


### **2.** 

### **Proposition:** <span style="color: var(--vscode-foreground); font-size: 14px;">Identify the most frequently occuring Pokemon for both the primary type and secondary type categories accross the three Pokemon generations and ranks the counts based on it and select the type with the highest occurance in each category.</span>

**Tables:**

- PokemonGen1.dbo.PokemonGen1
- PokemonGen2.dbo.PokemonGen2
- PokemonGen3.dbo.PokemonGen3

**Columns:**

- Type1
- Type2

**Predicate:**

- The query first aggregates the counts of each Pokemon for both Type1 and Type2 across all three generations in two separate CTE tables, Type1Counts and Type2Counts. It counts the number of occurances in the tables, excluding the null values. 
- Then it ranks the types based on the numbner of occurances using the RANK() function with two CTEs, RankType1 and RankType2
- The query then selects the most common types from RankedType1 and RankedType2 where the rank is 1 by using a SELECT statement with a UNION ALL
- If there are duplicates, or a case where the count is the same, all of them are diplayed

In [13]:
-- Calculate counts of Pokemon for each Type1 across all generations
WITH Type1Counts AS (
    SELECT Type1 AS Type, COUNT(*) AS Count
    FROM PokemonGen1.dbo.PokemonGen1
    WHERE Type1 IS NOT NULL
    GROUP BY Type1
    UNION ALL
    SELECT Type1, COUNT(*)
    FROM PokemonGen2.dbo.PokemonGen2
    WHERE Type1 IS NOT NULL
    GROUP BY Type1
    UNION ALL
    SELECT Type1, COUNT(*)
    FROM PokemonGen3.dbo.PokemonGen3
    WHERE Type1 IS NOT NULL
    GROUP BY Type1
),
-- Calculate counts of Pokemon for each Type2 across all generations
Type2Counts AS (
    SELECT Type2 AS Type, COUNT(*) AS Count
    FROM PokemonGen1.dbo.PokemonGen1
    WHERE Type2 IS NOT NULL
    GROUP BY Type2
    UNION ALL
    SELECT Type2, COUNT(*)
    FROM PokemonGen2.dbo.PokemonGen2
    WHERE Type2 IS NOT NULL
    GROUP BY Type2
    UNION ALL
    SELECT Type2, COUNT(*)
    FROM PokemonGen3.dbo.PokemonGen3
    WHERE Type2 IS NOT NULL
    GROUP BY Type2
),

-- Rank the Type1 counts
RankedType1 AS (
    SELECT Type, Count, RANK() OVER (ORDER BY Count DESC) AS Rank
    FROM Type1Counts
),

-- Rank the Type2 counts
RankedType2 AS (
    SELECT Type, Count, RANK() OVER (ORDER BY Count DESC) AS Rank
    FROM Type2Counts
)

-- Select the type with the highest count for each category (Type1 and Type2)
SELECT 'Type1' AS Category, Type, Count
FROM RankedType1
WHERE Rank = 1
UNION ALL
SELECT 'Type2' AS Category, Type, Count
FROM RankedType2
WHERE Rank = 1;


(4 rows affected)

Total execution time: 00:00:00.009

Category,Type,Count
Type1,Water,28
Type2,Flying,19
Type2,Poison,19
Type2,Flying,19


**3.**

**Proposition:** Write a query to retrieve a list of employees from the Northwinds2022TSQLV7 database who do not have an entry in the audit history, and also provide the department information for each employee, defaulting to 'NA' if the department is not specified.

**Tables:**

- HumanResources.Employee
- Triggered.AuditTriggeredEmployeeHistory
- SystemVersioned.Employee

**Columns:**

- EmployeeId
- EmployeeFirstName
- EmployeeLastName
- HireDate
- Department

**Predicate:**

- The NonAuditEmployees Common Table Expression (CTE) creates a subset of employees that don't have corresponding entries in the audit history table. The LEFT JOIN where the AuditTriggeredEmployeeHistory table has no matching EmployeeId (indicated by A.EmployeeId IS NULL)
- The main query selects the first name, last name, and hire date of the employees identified by the NonAuditEmployees CTE, then uses an INNER JOIN to ensure only the identified employee details are selected
- A LEFT JOIN to the SystemVersioned.Employee table tries to find department information for these employees. If the department is not found (which can happen because of the left join), COALESCE is used to default the value to 'NA'
- The ORDER BY clause ensures the resulting list is sorted alphabetically by the employee's last name and then by the first name

In [17]:
USE Northwinds2022TSQLV7;
-- Identify employees without audit records
WITH NonAuditEmployees AS (
    SELECT DISTINCT E.EmployeeId
    FROM [HumanResources].[Employee] E
    LEFT JOIN [Triggered].[AuditTriggeredEmployeeHistory] A ON E.EmployeeId = A.EmployeeId
    WHERE A.EmployeeId IS NULL
)
-- Select employee details including first name, last name, hire date, and department
SELECT
    E.EmployeeFirstName,
    E.EmployeeLastName,
    E.HireDate,
    COALESCE(SE.Department, 'NA') AS Department
FROM
    NonAuditEmployees NAE
    INNER JOIN [HumanResources].[Employee] E ON E.EmployeeId = NAE.EmployeeId
    LEFT JOIN [SystemVersioned].[Employee] SE ON SE.EmployeeId = E.EmployeeId
ORDER BY
    E.EmployeeLastName, 
    E.EmployeeFirstName;


(3 rows affected)

Total execution time: 00:00:00.006

EmployeeFirstName,EmployeeLastName,HireDate,Department
Maria,Cameron,2015-03-05,NA
Patricia,Doyle,2015-11-15,NA
Russell,King,2015-01-02,NA


## **Worst 3 Queries and Fixed Queries:**

**1.**

**Proposition:** Write a query using HR.Employees and Sales.Customers table to retrieve the country, region, and city using a CTE named EXCEPT\_ALL without any duplicates

**Tables:**

- <span style="color: var(--vscode-foreground);">HR.Employees (</span>HumanResources.\[Employee\])
    
- Sales.Customers (Sales.\[Customer\])
    

**Columns:**

- <span style="color: var(--vscode-foreground);">country (EmployeeCountry/CustomerCountry)</span>    
    
- region (EmployeeRegion/CustomerRegion)
    
- city (EmployeeCity/CustomerCity)
    

**Predicate:**

- The CTE is defined and named EXCEPT\_ALL and takes the rows from the HR.Employees table and removes whatever is duplicated based on the country, region, and city
- A row number is assigned to each group using the ROW\_NUMBER() function
- The PARTITION BY divides the rows based on the country, region, and city
- Then the ORDER BY clause is used to order the rows arbitrarily
- Then EXCEPT is used to remove rows from the Employees table that exist in the Sales.Customers table based on the country, region, and city
- The final SELECT statement retrieves the country, region and city from the EXCEPT\_ALL CTE

In [5]:
--Query using Northwinds2022TSQLV7
USE Northwinds2022TSQLV7;
WITH EXCEPT_ALL
AS
(
  SELECT
    ROW_NUMBER()
      OVER(PARTITION BY EmployeeCountry, EmployeeRegion, EmployeeCity
           ORDER     BY (SELECT 0)) AS rownum,
    EmployeeCountry, EmployeeRegion, EmployeeCity
  FROM HumanResources.[Employee]
  EXCEPT
  SELECT
    ROW_NUMBER()
      OVER(PARTITION BY CustomerCountry, CustomerRegion, CustomerCity
           ORDER     BY (SELECT 0)),
    CustomerCountry, CustomerRegion, CustomerCity
  FROM Sales.[Customer]
)
SELECT EmployeeCountry, EmployeeRegion, EmployeeCity
FROM EXCEPT_ALL;

(3 rows affected)

Total execution time: 00:00:00.029

EmployeeCountry,EmployeeRegion,EmployeeCity
USA,WA,Redmond
USA,WA,Tacoma
USA,WA,Seattle


### **Fixed query**

**Proposition:** Write a query that identifies the locations of where the company has employees but no customers and it counts the number of employees in each of the locations.

**Tables:**

- HumanResources.Employee: Contains data about the employees, specifically their location, country, region, city
- Sales.Customer: Contains data about the customers, including their location, country, region, city

**Columns:**

In the EmployeeLocations CTE:

- EmployeeCountry
- EmployeeRegion
- EmployeeCity

In the CustomerLocations CTE:

- CustomerCountry
- CustomerRegion
- CustomerCity

In the EmployeeCountInUniqueLocations CTE:

- EmployeeCountry
- EmployeeRegion
- EmployeeCity
- EmployeeCount

**Predicate:**

- The EmployeeLocations CTE is a virtual table that contains the distinct employee locations from the HumanResources.Employee table
- The CustomerLocations CTE is a virtual table that contains the distinct customer locations from the Sales.Customer table
- The UniqueEmployeeLocations CTE is a virtual table that is from subtracting the customer locations from employee locations and results in a set of locations where the company has employees but no customers
- The EmployeeCountInUniqueLocations CTE counts the number of employees in each of the locations from the UniqueEmployeesLocation
- The last SELECT statement then retrieves the country, region, and city with the count of the Employees

In [7]:
USE Northwinds2022TSQLV7;

WITH EmployeeLocations AS (
  SELECT
    EmployeeCountry,
    EmployeeRegion,
    EmployeeCity
  FROM HumanResources.Employee
),
CustomerLocations AS (
  SELECT
    CustomerCountry,
    CustomerRegion,
    CustomerCity
  FROM Sales.Customer
),
UniqueEmployeeLocations AS (
  SELECT
    EmployeeCountry, EmployeeRegion, EmployeeCity
  FROM EmployeeLocations

  EXCEPT

  SELECT
    CustomerCountry, CustomerRegion, CustomerCity
  FROM CustomerLocations
),
EmployeeCountInUniqueLocations AS (
  SELECT
    EmployeeCountry,
    EmployeeRegion,
    EmployeeCity,
    COUNT(*) AS EmployeeCount
  FROM HumanResources.Employee e
  WHERE EXISTS (
    SELECT 1
    FROM UniqueEmployeeLocations u
    WHERE u.EmployeeCountry = e.EmployeeCountry
      AND u.EmployeeRegion = e.EmployeeRegion
      AND u.EmployeeCity = e.EmployeeCity
  )
  GROUP BY EmployeeCountry, EmployeeRegion, EmployeeCity
)
SELECT
  EmployeeCountry,
  EmployeeRegion,
  EmployeeCity,
  EmployeeCount
FROM EmployeeCountInUniqueLocations
ORDER BY EmployeeCountry, EmployeeRegion, EmployeeCity;


(2 rows affected)

Total execution time: 00:00:00.011

EmployeeCountry,EmployeeRegion,EmployeeCity,EmployeeCount
USA,WA,Redmond,1
USA,WA,Tacoma,1


**2.**

**Proposition:** Write a query to retrive the combinations of country, region, and city from Production.Suppliers table that are not in HR.Employees table but coincide with the entries in Sales.Customers table using the EXCEPT and INTERSECT operator

**Tables:**

- Production.Suppliers (Production.\[Supplier\])
- HR.Employees (HumanResources.\[Employee\])
- Sales.Customers (Sales.\[Customer\])

**Columns:**

- country (SupplierCountry/EmployeeCountry/CustomerCountry)
- region (SupplierRegion/EmployeeRegion/CustomerRegion)
- city (CustomerCountry/CustomerRegion/CustomerCity)

**Predicate:**

- EXCEPT operation is performed between the country, region, and city from Production.Suppliers and the HR.Employees table
- Then INTERSECT between the results from the EXCEPT operation of the country, region, and city from the Sales.Customers table
- The parentheses is used to specify the order of operations to ensure that EXCEPT is done before the INTERSECT operation. 
- The query results in the country, region, and city from the Productions.Suppliers table that are not in HR.Employees table but intersect with the Sales.Customers table

In [6]:
--Query using Northwinds2022TSQLV7
USE Northwinds2022TSQLV7;
SELECT SupplierCountry, SupplierRegion, SupplierCity FROM Production.[Supplier]
EXCEPT
SELECT EmployeeCountry, EmployeeRegion, EmployeeCity FROM HumanResources.[Employee]
INTERSECT
SELECT CustomerCountry, CustomerRegion, CustomerCity FROM Sales.[Customer];

(SELECT SupplierCountry, SupplierRegion, SupplierCity FROM Production.[Supplier]
 EXCEPT
 SELECT EmployeeCountry, EmployeeRegion, EmployeeCity FROM HumanResources.[Employee])
INTERSECT
SELECT CustomerCountry, CustomerRegion, CustomerCity FROM Sales.[Customer];

(28 rows affected)

(3 rows affected)

Total execution time: 00:00:00.034

SupplierCountry,SupplierRegion,SupplierCity
Australia,NSW,Sydney
Australia,Victoria,Melbourne
Brazil,NULL,Sao Paulo
Canada,Québec,Montréal
Canada,Québec,Ste-Hyacinthe
Denmark,NULL,Lyngby
Finland,NULL,Lappeenranta
France,NULL,Annecy
France,NULL,Montceau
France,NULL,Paris


SupplierCountry,SupplierRegion,SupplierCity
Canada,Québec,Montréal
France,NULL,Paris
Germany,NULL,Berlin


### **Fixed Query**

**Proposition:** Write a query to find the <span style="color: var(--vscode-foreground);">locations where there are suppliers that do not overlap with employee locations but overlap with customer locations. For the unique locations</span><span style="color: var(--vscode-foreground);">, it counts the number of suppliers and presents the locations sorted by the number of suppliers in descending order, and then by country, region, and city</span>

**Tables:**

- Production.Supplier
- HumanResources.Employee
- Sales.Customer

**Columns:**

- SupplierCountry, SupplierRegion, SupplierCity
- EmployeeCountry, EmployeeRegion, EmployeeCity
- CustomerCountry, CustomerRegion, CustomerCity
- NumberOfSuppliers

**Predicate:**

- SupplierLocations CTE extracts the location details of suppliers
- EmployeeLocations CTE extracts the location details of employees
- CustomerLocations CTE extracts the location details of customers
- UniqueSupplierLocations CTE finds the locations where suppliers exist but employees do not (SELECT FROM SupplierLocations EXCEPT SELECT FROM EmployeeLocations)
- Then it intersects these results with the locations where customers exist (INTERSECT SELECT FROM CustomerLocations), this ensures the locations are unique to suppliers when compared to employee locations but also exist in customer locations
- SuppliersCount CTE has a unique location identified in the previous step and counts the number of suppliers.
- Finally the SELECT operator outputs the locations and the count of suppliers and sorts it by the number of suppliers in descending order and then by the location details

In [8]:
USE Northwinds2022TSQLV7;
WITH SupplierLocations AS (
  SELECT
    SupplierCountry,
    SupplierRegion,
    SupplierCity
  FROM Production.Supplier
),
EmployeeLocations AS (
  SELECT
    EmployeeCountry,
    EmployeeRegion,
    EmployeeCity
  FROM HumanResources.Employee
),
CustomerLocations AS (
  SELECT
    CustomerCountry,
    CustomerRegion,
    CustomerCity
  FROM Sales.Customer
),
UniqueSupplierLocations AS (
  (SELECT SupplierCountry, SupplierRegion, SupplierCity FROM SupplierLocations
   EXCEPT
   SELECT EmployeeCountry, EmployeeRegion, EmployeeCity FROM EmployeeLocations)
  INTERSECT
  SELECT CustomerCountry, CustomerRegion, CustomerCity FROM CustomerLocations
),
SuppliersCount AS (
  SELECT
    u.SupplierCountry,
    u.SupplierRegion,
    u.SupplierCity,
    COUNT(*) AS NumberOfSuppliers
  FROM Production.Supplier s
  INNER JOIN UniqueSupplierLocations u ON s.SupplierCountry = u.SupplierCountry
                                       AND s.SupplierRegion = u.SupplierRegion
                                       AND s.SupplierCity = u.SupplierCity
  GROUP BY u.SupplierCountry, u.SupplierRegion, u.SupplierCity
)
SELECT
  SupplierCountry,
  SupplierRegion,
  SupplierCity,
  NumberOfSuppliers
FROM SuppliersCount
ORDER BY NumberOfSuppliers DESC, SupplierCountry, SupplierRegion, SupplierCity;


(1 row affected)

Total execution time: 00:00:00.021

SupplierCountry,SupplierRegion,SupplierCity,NumberOfSuppliers
Canada,Québec,Montréal,1


### **3.**

**Proposition:** Write a query that  provides a report that includes each department's name, the count of active employees in that department, and the average pay of those employees. Departments with no active employees will still be listed with a count of zero and an average pay of zero

**Tables:**

- HumanResources.EmployeePayHistory (aliased as ep)
- HumanResources.EmployeeDepartmentHistory (aliased as edh)
- HumanResources.Department (aliased as d)

**Columns:**

- edh.DepartmentID
- ep.Rate
- d.Name
- edh.BusinessEntityID

**Predicate:**

- DepartmentPaySummary CTE calculates the average pay for active employees in each department by joining EmployeePayHistory and EmployeeDepartmentHistory on the common key BusinessEntityID and filtering on edh.EndDate IS NULL to include only current employee assignments
- Department table to EmployeeDepartmentHistory on DepartmentID are joined to departments with their current employees.
- LEFT JOIN to the DepartmentPaySummary includes the average pay data in the results
- The COALESCE function ensures that departments without pay data (due to having no active employees) show an average pay of zero rather than null.
- The WHERE edh.EndDate IS NULL condition makes sure that only current employees are considered
- The GROUP BY clause groups the results by department name and average pay
- The ORDER BY clause sorts the results first by the number of employees in descending order and then by average pay in descending order

In [11]:
USE AdventureWorks2017;
WITH DepartmentPaySummary AS (
    SELECT
        edh.DepartmentID,
        AVG(ep.Rate) AS AveragePay 
    FROM
        HumanResources.EmployeePayHistory ep
        INNER JOIN HumanResources.EmployeeDepartmentHistory edh ON ep.BusinessEntityID = edh.BusinessEntityID
    WHERE
        edh.EndDate IS NULL
    GROUP BY
        edh.DepartmentID
)
SELECT
    d.Name AS DepartmentName,
    COUNT(edh.BusinessEntityID) AS NumberOfEmployees,
    COALESCE(dps.AveragePay, 0) AS AverageDepartmentPay
FROM
    HumanResources.Department d
    INNER JOIN HumanResources.EmployeeDepartmentHistory edh ON d.DepartmentID = edh.DepartmentID
    LEFT JOIN DepartmentPaySummary dps ON d.DepartmentID = dps.DepartmentID
WHERE
    edh.EndDate IS NULL
GROUP BY
    d.Name,
    dps.AveragePay
ORDER BY
    NumberOfEmployees DESC, 
    AverageDepartmentPay DESC;

(16 rows affected)

Total execution time: 00:00:00.023

DepartmentName,NumberOfEmployees,AverageDepartmentPay
Production,179,13.5537
Sales,18,29.9719
Purchasing,12,18.0202
Information Services,10,34.1586
Finance,10,23.935
Marketing,9,18.4318
Facilities and Maintenance,7,13.0316
Engineering,6,40.1442
Human Resources,6,18.0248
Production Control,6,16.7746


### **Fixed Query**

**Proposition:** Write a query that provides an overview of current departmental pay, listing each department along with the number of active employees and their average pay rate.

**Tables:**

- HumanResources.EmployeePayHistory
- HumanResources.EmployeeDepartmentHistory
- HumanResources.Department

**Columns:**

- DepartmentName
- NumberOfEmployees
- AverageDepartmentPay

**Predicate:**

- <span style="color: var(--vscode-foreground);">Only including employee records where the EndDate is null in EmployeeDepartmentHistory, which means they are current employees and group it by the department name</span>
- Then ordering by the number of employees in descending order, then by average department pay in descending order

In [10]:
USE AdventureWorks2017;
WITH CurrentDepartmentPay AS (
    SELECT
        edh.BusinessEntityID,
        edh.DepartmentID,
        ep.Rate,
        ROW_NUMBER() OVER(PARTITION BY ep.BusinessEntityID ORDER BY ep.RateChangeDate DESC) AS rn
    FROM
        HumanResources.EmployeePayHistory ep
        INNER JOIN HumanResources.EmployeeDepartmentHistory edh ON ep.BusinessEntityID = edh.BusinessEntityID
    WHERE
        edh.EndDate IS NULL
)
SELECT
    d.Name AS DepartmentName,
    COUNT(distinct cdp.BusinessEntityID) AS NumberOfEmployees,
    MAX(cdp.Rate) AS MaxSalary,
    MIN(cdp.Rate) AS MinSalary
FROM
    HumanResources.Department d
    INNER JOIN CurrentDepartmentPay cdp ON d.DepartmentID = cdp.DepartmentID
    INNER JOIN HumanResources.Employee e ON cdp.BusinessEntityID = e.BusinessEntityID
WHERE
    cdp.rn = 1 
GROUP BY
    d.Name
ORDER BY
    NumberOfEmployees DESC;


(16 rows affected)

Total execution time: 00:00:00.023

DepartmentName,NumberOfEmployees,MaxSalary,MinSalary
Production,179,84.1346,9.50
Sales,18,72.1154,23.0769
Purchasing,12,30.00,12.75
Finance,10,43.2692,13.4615
Information Services,10,50.4808,27.4038
Marketing,9,37.50,13.4615
Facilities and Maintenance,7,24.0385,9.25
Engineering,6,63.4615,32.6923
Human Resources,6,27.1394,13.9423
Quality Assurance,6,28.8462,10.5769


## **Medium Queries**

**Query 1**

**Proposition:** Write a query using HR.Employees and Sales.Customers table to retrieve the country, region, and city using a CTE named EXCEPT\_ALL without any duplicates

**Tables:**

- HR.Employees (HumanResources.\[Employee\])
    
- Sales.Customers (Sales.\[Customer\])
    

**Columns:**

- country (EmployeeCountry/CustomerCountry)    
    
- region (EmployeeRegion/CustomerRegion)
    
- city (EmployeeCity/CustomerCity)
    

**Predicate:**

- The CTE is defined and named EXCEPT\_ALL and takes the rows from the HR.Employees table and removes whatever is duplicated based on the country, region, and city
- A row number is assigned to each group using the ROW\_NUMBER() function
- The PARTITION BY divides the rows based on the country, region, and city
- Then the ORDER BY clause is used to order the rows arbitrarily
- Then EXCEPT is used to remove rows from the Employees table that exist in the Sales.Customers table based on the country, region, and city
- The final SELECT statement retrieves the country, region and city from the EXCEPT\_ALL CTE

In [9]:
USE Northwinds2022TSQLV7;
WITH EXCEPT_ALL
AS
(
  SELECT
    ROW_NUMBER()
      OVER(PARTITION BY EmployeeCountry, EmployeeRegion, EmployeeCity
           ORDER     BY (SELECT 0)) AS rownum,
    EmployeeCountry, EmployeeRegion, EmployeeCity
  FROM HumanResources.[Employee]
  EXCEPT
  SELECT
    ROW_NUMBER()
      OVER(PARTITION BY CustomerCountry, CustomerRegion, CustomerCity
           ORDER     BY (SELECT 0)),
    CustomerCountry, CustomerRegion, CustomerCity
  FROM Sales.[Customer]
)
SELECT EmployeeCountry, EmployeeRegion, EmployeeCity
FROM EXCEPT_ALL;

(3 rows affected)

Total execution time: 00:00:00.015

EmployeeCountry,EmployeeRegion,EmployeeCity
USA,WA,Redmond
USA,WA,Tacoma
USA,WA,Seattle


**Query 2**

**Proposition:** Write a query to retrive the combinations of country, region, and city from Production.Suppliers table that are not in HR.Employees table but coincide with the entries in Sales.Customers table using the EXCEPT and INTERSECT operator

**Tables:**

- Production.Suppliers (Production.\[Supplier\])
- HR.Employees (HumanResources.\[Employee\])
- Sales.Customers (Sales.\[Customer\])

**Columns:**

- country (SupplierCountry/EmployeeCountry/CustomerCountry)
- region (SupplierRegion/EmployeeRegion/CustomerRegion)
- city (CustomerCountry/CustomerRegion/CustomerCity)

**Predicate:**

- EXCEPT operation is performed between the country, region, and city from Production.Suppliers and the HR.Employees table
- Then INTERSECT between the results from the EXCEPT operation of the country, region, and city from the Sales.Customers table
- The parentheses is used to specify the order of operations to ensure that EXCEPT is done before the INTERSECT operation. 
- The query results in the country, region, and city from the Productions.Suppliers table that are not in HR.Employees table but intersect with the Sales.Customers table

In [9]:
USE Northwinds2022TSQLV7;
SELECT SupplierCountry, SupplierRegion, SupplierCity FROM Production.[Supplier]
EXCEPT
SELECT EmployeeCountry, EmployeeRegion, EmployeeCity FROM HumanResources.[Employee]
INTERSECT
SELECT CustomerCountry, CustomerRegion, CustomerCity FROM Sales.[Customer];

(SELECT SupplierCountry, SupplierRegion, SupplierCity FROM Production.[Supplier]
 EXCEPT
 SELECT EmployeeCountry, EmployeeRegion, EmployeeCity FROM HumanResources.[Employee])
INTERSECT
SELECT CustomerCountry, CustomerRegion, CustomerCity FROM Sales.[Customer];

(28 rows affected)

(3 rows affected)

Total execution time: 00:00:00.018

SupplierCountry,SupplierRegion,SupplierCity
Australia,NSW,Sydney
Australia,Victoria,Melbourne
Brazil,NULL,Sao Paulo
Canada,Québec,Montréal
Canada,Québec,Ste-Hyacinthe
Denmark,NULL,Lyngby
Finland,NULL,Lappeenranta
France,NULL,Annecy
France,NULL,Montceau
France,NULL,Paris


SupplierCountry,SupplierRegion,SupplierCity
Canada,Québec,Montréal
France,NULL,Paris
Germany,NULL,Berlin


**Query 3**

**Proposition:** Write a query to retrieve a list of employees from the Northwinds2022TSQLV7 database who do not have an entry in the audit history, and also provide the department information for each employee, defaulting to 'NA' if the department is not specified.

**Tables:**

- HumanResources.Employee
- Triggered.AuditTriggeredEmployeeHistory
- SystemVersioned.Employee

**Columns:**

- EmployeeId
- EmployeeFirstName
- EmployeeLastName
- HireDate
- Department

**Predicate:**

- The NonAuditEmployees Common Table Expression (CTE) creates a subset of employees that don't have corresponding entries in the audit history table. The LEFT JOIN where the AuditTriggeredEmployeeHistory table has no matching EmployeeId (indicated by A.EmployeeId IS NULL)
- The main query selects the first name, last name, and hire date of the employees identified by the NonAuditEmployees CTE, then uses an INNER JOIN to ensure only the identified employee details are selected
- A LEFT JOIN to the SystemVersioned.Employee table tries to find department information for these employees. If the department is not found (which can happen because of the left join), COALESCE is used to default the value to 'NA'
- The ORDER BY clause ensures the resulting list is sorted alphabetically by the employee's last name and then by the first name

In [5]:
USE Northwinds2022TSQLV7;
WITH NonAuditEmployees AS (
    SELECT DISTINCT E.EmployeeId
    FROM [HumanResources].[Employee] E
    LEFT JOIN [Triggered].[AuditTriggeredEmployeeHistory] A ON E.EmployeeId = A.EmployeeId
    WHERE A.EmployeeId IS NULL
)
SELECT
    E.EmployeeFirstName,
    E.EmployeeLastName,
    E.HireDate,
    COALESCE(SE.Department, 'NA') AS Department
FROM
    NonAuditEmployees NAE
    INNER JOIN [HumanResources].[Employee] E ON E.EmployeeId = NAE.EmployeeId
    LEFT JOIN [SystemVersioned].[Employee] SE ON SE.EmployeeId = E.EmployeeId
ORDER BY
    E.EmployeeLastName, 
    E.EmployeeFirstName;

(3 rows affected)

Total execution time: 00:00:00.017

EmployeeFirstName,EmployeeLastName,HireDate,Department
Maria,Cameron,2015-03-05,NA
Patricia,Doyle,2015-11-15,NA
Russell,King,2015-01-02,NA


### **Query 4**

**Proposition:** Write a query that  provides a report that includes each department's name, the count of active employees in that department, and the average pay of those employees. Departments with no active employees will still be listed with a count of zero and an average pay of zero

**Tables:**

- HumanResources.EmployeePayHistory (aliased as ep)
- HumanResources.EmployeeDepartmentHistory (aliased as edh)
- HumanResources.Department (aliased as d)

**Columns:**

- edh.DepartmentID
- ep.Rate
- d.Name
- edh.BusinessEntityID

**Predicate:**

- DepartmentPaySummary CTE calculates the average pay for active employees in each department by joining EmployeePayHistory and EmployeeDepartmentHistory on the common key BusinessEntityID and filtering on edh.EndDate IS NULL to include only current employee assignments
- Department table to EmployeeDepartmentHistory on DepartmentID are joined to departments with their current employees.
- LEFT JOIN to the DepartmentPaySummary includes the average pay data in the results
- The COALESCE function ensures that departments without pay data (due to having no active employees) show an average pay of zero rather than null.
- The WHERE edh.EndDate IS NULL condition makes sure that only current employees are considered
- The GROUP BY clause groups the results by department name and average pay
- The ORDER BY clause sorts the results first by the number of employees in descending order and then by average pay in descending order

In [8]:
USE AdventureWorks2017;
WITH DepartmentPaySummary AS (
    SELECT
        edh.DepartmentID,
        AVG(ep.Rate) AS AveragePay 
    FROM
        HumanResources.EmployeePayHistory ep
        INNER JOIN HumanResources.EmployeeDepartmentHistory edh ON ep.BusinessEntityID = edh.BusinessEntityID
    WHERE
        edh.EndDate IS NULL
    GROUP BY
        edh.DepartmentID
)
SELECT
    d.Name AS DepartmentName,
    COUNT(edh.BusinessEntityID) AS NumberOfEmployees,
    COALESCE(dps.AveragePay, 0) AS AverageDepartmentPay
FROM
    HumanResources.Department d
    INNER JOIN HumanResources.EmployeeDepartmentHistory edh ON d.DepartmentID = edh.DepartmentID
    LEFT JOIN DepartmentPaySummary dps ON d.DepartmentID = dps.DepartmentID
WHERE
    edh.EndDate IS NULL
GROUP BY
    d.Name,
    dps.AveragePay
ORDER BY
    NumberOfEmployees DESC, 
    AverageDepartmentPay DESC;


(16 rows affected)

Total execution time: 00:00:00.016

DepartmentName,NumberOfEmployees,AverageDepartmentPay
Production,179,13.5537
Sales,18,29.9719
Purchasing,12,18.0202
Information Services,10,34.1586
Finance,10,23.935
Marketing,9,18.4318
Facilities and Maintenance,7,13.0316
Engineering,6,40.1442
Human Resources,6,18.0248
Production Control,6,16.7746


### **Query 5**

**Proposition:** Write a query that provides an overview of current departmental pay, listing each department along with the number of active employees and their average pay rate.

**Tables:**

- HumanResources.EmployeePayHistory
- HumanResources.EmployeeDepartmentHistory
- HumanResources.Department

**Columns:**

- DepartmentName
- NumberOfEmployees
- AverageDepartmentPay

**Predicate:**

- <span style="color: var(--vscode-foreground);">Only including employee records where the EndDate is null in EmployeeDepartmentHistory, which means they are current employees and group it by the department name</span>
- Then ordering by the number of employees in descending order, then by average department pay in descending order

In [2]:
USE AdventureWorks2017;
WITH CurrentDepartmentPay AS (
    SELECT
        edh.BusinessEntityID,
        edh.DepartmentID,
        ep.Rate,
        ROW_NUMBER() OVER(PARTITION BY ep.BusinessEntityID ORDER BY ep.RateChangeDate DESC) AS rn
    FROM
        HumanResources.EmployeePayHistory ep
        INNER JOIN HumanResources.EmployeeDepartmentHistory edh ON ep.BusinessEntityID = edh.BusinessEntityID
    WHERE
        edh.EndDate IS NULL
)
SELECT
    d.Name AS DepartmentName,
    COUNT(distinct cdp.BusinessEntityID) AS NumberOfEmployees,
    MAX(cdp.Rate) AS MaxSalary,
    MIN(cdp.Rate) AS MinSalary
FROM
    HumanResources.Department d
    INNER JOIN CurrentDepartmentPay cdp ON d.DepartmentID = cdp.DepartmentID
    INNER JOIN HumanResources.Employee e ON cdp.BusinessEntityID = e.BusinessEntityID
WHERE
    cdp.rn = 1 
GROUP BY
    d.Name
ORDER BY
    NumberOfEmployees DESC;


(16 rows affected)

Total execution time: 00:00:00.125

DepartmentName,NumberOfEmployees,MaxSalary,MinSalary
Production,179,84.1346,9.50
Sales,18,72.1154,23.0769
Purchasing,12,30.00,12.75
Finance,10,43.2692,13.4615
Information Services,10,50.4808,27.4038
Marketing,9,37.50,13.4615
Facilities and Maintenance,7,24.0385,9.25
Engineering,6,63.4615,32.6923
Human Resources,6,27.1394,13.9423
Quality Assurance,6,28.8462,10.5769


### **Query 6**

**Proposition:** Write a query that summarizes total sales by product category, including the total quantity sold and the total sales amount

**Tables:**

- Production.Product
- Production.ProductSubcategory
- Production.ProductCategory
- Sales.SalesOrderDetail
- Sales.SalesOrderHeader

**Columns:** 

- ProductCategory 
- TotalQuantitySold 
- TotalSales

**Predicate:**

- Grouping by the name of the product category
- Ordering by the total sales in descending order

In [3]:
USE AdventureWorks2017;
SELECT
    pc.Name AS ProductCategory,
    SUM(sod.OrderQty) AS TotalQuantitySold,
    SUM(sod.LineTotal) AS TotalSales
FROM
    Production.Product p
    INNER JOIN Sales.SalesOrderDetail sod ON p.ProductID = sod.ProductID
    INNER JOIN Production.ProductSubcategory ps ON p.ProductSubcategoryID = ps.ProductSubcategoryID
    INNER JOIN Production.ProductCategory pc ON ps.ProductCategoryID = pc.ProductCategoryID
    INNER JOIN Sales.SalesOrderHeader soh ON sod.SalesOrderID = soh.SalesOrderID
GROUP BY
    pc.Name
ORDER BY
    TotalSales DESC;

(4 rows affected)

Total execution time: 00:00:00.297

ProductCategory,TotalQuantitySold,TotalSales
Bikes,90268,94651172.704731
Components,49044,11802593.286430
Clothing,73670,2120542.524801
Accessories,61932,1272072.883926


### **Query 7**

**Proposition:** Write a query that <span style="color: var(--vscode-foreground);">classifies customers as 'Individual', 'Store', or 'Other' and summarizes the number of orders and total sales amount for each customer type.</span>

**Tables:**

- Sales.Customer
- Sales.SalesOrderHeader
- Sales.SalesOrderDetail

**Columns:**

- CustomerType
- NumberOfOrders 
- TotalSalesAmount

**Predicate:**

- Grouping by the customer type
- Ordering by the total sales amount in descending order

In [7]:
USE AdventureWorks2017;
SELECT
    CASE
        WHEN C.PersonID IS NOT NULL THEN 'Individual'
        WHEN C.StoreID IS NOT NULL THEN 'Store'
        ELSE 'Other'
    END AS CustomerType,
    COUNT(DISTINCT SOH.SalesOrderID) AS NumberOfOrders,
    SUM(SOD.LineTotal) AS TotalSalesAmount
FROM
    Sales.Customer AS C
    INNER JOIN Sales.SalesOrderHeader AS SOH ON C.CustomerID = SOH.CustomerID
    INNER JOIN Sales.SalesOrderDetail AS SOD ON SOH.SalesOrderID = SOD.SalesOrderID
GROUP BY
    CASE
        WHEN C.PersonID IS NOT NULL THEN 'Individual'
        WHEN C.StoreID IS NOT NULL THEN 'Store'
        ELSE 'Other'
    END
ORDER BY
    TotalSalesAmount DESC;


(1 row affected)

Total execution time: 00:00:00.270

CustomerType,NumberOfOrders,TotalSalesAmount
Individual,31465,109846381.399888


### **Query 8**

**Proposition:**

The query calculates the number of orders, total sales amount, and average discount applied for each sales territory

**Tables:**

- Sales.SalesOrderHeader
- Sales.SalesOrderDetail
- Sales.SalesTerritory

**Columns:**

- TerritoryName  
- NumberOfOrders  
- TotalSalesAmount 
- AverageDiscount 

**Predicate:**

- Grouping by sales territory name
- Ordering by the total sales amount in descending order

In [5]:
USE AdventureWorks2017;
SELECT
    ST.Name AS TerritoryName,
    COUNT(DISTINCT SOH.SalesOrderID) AS NumberOfOrders,
    SUM(SOD.LineTotal) AS TotalSalesAmount,
    AVG(SOD.UnitPriceDiscount) AS AverageDiscount
FROM
    Sales.SalesOrderHeader AS SOH
    INNER JOIN Sales.SalesOrderDetail AS SOD ON SOH.SalesOrderID = SOD.SalesOrderID
    INNER JOIN Sales.SalesTerritory AS ST ON SOH.TerritoryID = ST.TerritoryID
GROUP BY
    ST.Name
ORDER BY
    TotalSalesAmount DESC;


(10 rows affected)

Total execution time: 00:00:00.326

TerritoryName,NumberOfOrders,TotalSalesAmount,AverageDiscount
Southwest,6224,24184609.600810,0.0026
Canada,4067,16355770.454862,0.0028
Northwest,4594,16084942.547585,0.0031
Australia,6843,10655335.959317,0.0014
Central,385,7909009.005872,0.0053
Southeast,486,7879655.072151,0.005
United Kingdom,3219,7670721.035475,0.0021
France,2672,7251555.646926,0.0023
Northeast,352,6939374.481005,0.0036
Germany,2623,4915407.595885,0.0026


### **Query 9**

**Proposition:** Write a query that <span style="color: var(--vscode-foreground);">provides a summary of purchases from vendors, including the number of purchase orders, total quantity ordered, and average unit price of items ordered</span>

**Tables:**

- Purchasing.PurchaseOrderHeader
- Purchasing.PurchaseOrderDetail
- Purchasing.Vendor

**Columns:**

- VendorName (from Purchasing.Vendor)
- NumberOfPurchaseOrders (the count of distinct purchase order IDs in Purchasing.PurchaseOrderHeader)
- TotalQuantityOrdered (the sum of order quantity in Purchasing.PurchaseOrderDetail)
- AverageUnitPrice (the average of unit price in Purchasing.PurchaseOrderDetail)

**Predicate:**

- Grouping by vendor name
- Ordering by the total quantity ordered in descending order

In [6]:
USE AdventureWorks2017;
SELECT
    V.Name AS VendorName,
    COUNT(DISTINCT POH.PurchaseOrderID) AS NumberOfPurchaseOrders,
    SUM(POD.OrderQty) AS TotalQuantityOrdered,
    AVG(POD.UnitPrice) AS AverageUnitPrice
FROM
    Purchasing.PurchaseOrderHeader AS POH
    INNER JOIN Purchasing.PurchaseOrderDetail AS POD ON POH.PurchaseOrderID = POD.PurchaseOrderID
    INNER JOIN Purchasing.Vendor AS V ON POH.VendorID = V.BusinessEntityID
GROUP BY
    V.Name
ORDER BY
    TotalQuantityOrdered DESC;


(86 rows affected)

Total execution time: 00:00:00.143

VendorName,NumberOfPurchaseOrders,TotalQuantityOrdered,AverageUnitPrice
SUPERSALES INC.,50,125000,0.21
"Custom Frames, Inc.",51,115500,10.3072
Chicago City Saddles,51,98450,30.7679
Victory Bikes,50,79200,23.4491
Professional Athletic Consultants,50,78100,39.1648
"Compete Enterprises, Inc",51,69300,22.6135
Circuit Cycles,51,69300,4.235
"Compete, Inc.",51,68750,4.6074
First Rate Bicycles,51,67650,30.8245
Electronic Bike Repair & Supplies,51,67100,29.0614


### **Query 10**

**Proposition:** Write a query to show the sales performance of each sales territory in terms of the total sales volume, the number of orders, and the average order size.

**Tables:**

- Sales.SalesOrderHeader
- Sales.SalesTerritory

**Columns:**

- TerritoryName  
- OrderCount 
- TotalSales 
- AverageOrderSize

**Predicate:**

- The query joins the SalesOrderHeader table with the SalesTerritory table to relate orders to their respective sales territories
- Then results are grouped by the name of the territory
- Orders are counted, and sales are totaled and averaged by territory
- The results are ordered by the total sales in descending order to show the territories that have the highest sales volumes

In [10]:
USE AdventureWorks2017;
SELECT
    ST.Name AS TerritoryName,
    COUNT(SOH.SalesOrderID) AS OrderCount,
    SUM(SOH.TotalDue) AS TotalSales,
    AVG(SOH.TotalDue) AS AverageOrderSize
FROM
    Sales.SalesOrderHeader SOH
    INNER JOIN Sales.SalesTerritory ST ON SOH.TerritoryID = ST.TerritoryID
GROUP BY
    ST.Name
ORDER BY
    TotalSales DESC;


(10 rows affected)

Total execution time: 00:00:00.035

TerritoryName,OrderCount,TotalSales,AverageOrderSize
Southwest,6224,27150594.5893,4362.242
Canada,4067,18398929.188,4523.956
Northwest,4594,18061660.371,3931.576
Australia,6843,11814376.0952,1726.4907
Central,385,8913299.2473,23151.4266
Southeast,486,8884099.3669,18280.0398
United Kingdom,3219,8574048.7082,2663.5752
France,2672,8119749.346,3038.8283
Northeast,352,7820209.6285,22216.5046
Germany,2623,5479819.5755,2089.142


### **Query 11**

**Proposition:** <span style="color: var(--vscode-foreground);">Write a query that aims to summarize the total number of purchase orders, the total quantity of items ordered, and the total cost of purchases for each vendor.</span>

**Tables:**

- Purchasing.PurchaseOrderDetail
- Purchasing.PurchaseOrderHeader
- Purchasing.Vendor

**Columns:**

- VendorName  
- TotalPurchaseOrders  
- TotalQuantity  
- TotalCost  

**Predicate:**

- The query joins the PurchaseOrderDetail table with PurchaseOrderHeader and then with Vendor to categorize purchases and are grouped by the vendor name
- It then counts the total number of purchase orders, sums the total quantity of items ordered, and calculates the total cost for each vendor and is put in descending order

In [11]:
USE AdventureWorks2017;

SELECT
    V.Name AS VendorName,
    COUNT(POD.PurchaseOrderID) AS TotalPurchaseOrders,
    SUM(POD.OrderQty) AS TotalQuantity,
    SUM(POD.LineTotal) AS TotalCost
FROM
    Purchasing.PurchaseOrderDetail POD
    INNER JOIN Purchasing.PurchaseOrderHeader POH ON POD.PurchaseOrderID = POH.PurchaseOrderID
    INNER JOIN Purchasing.Vendor V ON POH.VendorID = V.BusinessEntityID
GROUP BY
    V.Name
ORDER BY
    TotalCost DESC;


(86 rows affected)

Total execution time: 00:00:00.042

VendorName,TotalPurchaseOrders,TotalQuantity,TotalCost
Superior Bicycles,100,55000,4555897.50
Professional Athletic Consultants,142,78100,3058774.95
Chicago City Saddles,179,98450,3029108.775
Jackson Authority,120,66000,2553243.00
"Vision Cycles, Inc.",120,66000,2513742.00
Sport Fan Co.,117,64350,2421619.20
"Proseware, Inc.",120,66000,2347422.00
Greenwood Athletic Company,102,56100,2237800.95
Crowley Sport,102,56100,2237800.95
Mitchell Sports,100,55000,2193922.50


### **Query 12**

**Proposition:** Write a query to report the inventory quantities of products at various locations, summarizing the total quantity per product per location.

**Tables:**

- Production.ProductInventory
- Production.Product
- Production.Location

**Columns:**

- LocationName  
- ProductName 
- TotalQuantity  

**Predicate:**

- The query joins the ProductInventory table with the Product and Location tables to correlate inventory records with product names and location name and the results are grouped by location and product names and are location name and total quantity in descending order, showing the highest stocked items first.

In [12]:
USE AdventureWorks2017;

SELECT
    L.Name AS LocationName,
    P.Name AS ProductName,
    SUM(PI.Quantity) AS TotalQuantity
FROM
    Production.ProductInventory PI
    INNER JOIN Production.Product P ON PI.ProductID = P.ProductID
    INNER JOIN Production.Location L ON PI.LocationID = L.LocationID
GROUP BY
    L.Name,
    P.Name
ORDER BY
    L.Name,
    TotalQuantity DESC;


(1069 rows affected)

Total execution time: 00:00:00.031

LocationName,ProductName,TotalQuantity
Debur and Polish,"HL Mountain Frame - Black, 38",148
Debur and Polish,"ML Mountain Frame-W - Silver, 38",147
Debur and Polish,"ML Road Frame-W - Yellow, 38",142
Debur and Polish,"ML Mountain Frame - Black, 38",137
Debur and Polish,"HL Mountain Frame - Silver, 38",132
Debur and Polish,"LL Mountain Frame - Black, 40",129
Debur and Polish,"LL Mountain Frame - Silver, 40",123
Final Assembly,Seat Lug,729
Final Assembly,Decal 1,641
Final Assembly,Flat Washer 3,641


### **Query 13**

**Proposition:** Write a <span style="color: var(--vscode-foreground);">query to display the list price history of products, showing any changes in pricing over time for each product.</span>

**Tables:**

- Production.Product
- Production.ProductListPriceHistory

**Columns:**

- ProductName 
- StartDate 
- EndDate  
- ListPrice 

**Predicate:**

- The query joins the Product table with ProductListPriceHistory to associate products with their price histories
- Results are ordered by product name and start date in descending order, which shows the most recent price changes first

In [13]:
USE AdventureWorks2017;
SELECT
    P.Name AS ProductName,
    PLPH.StartDate,
    PLPH.EndDate,
    PLPH.ListPrice
FROM
    Production.Product P
    INNER JOIN Production.ProductListPriceHistory PLPH ON P.ProductID = PLPH.ProductID
ORDER BY
    P.Name,
    PLPH.StartDate DESC;


(395 rows affected)

Total execution time: 00:00:00.029

ProductName,StartDate,EndDate,ListPrice
All-Purpose Bike Stand,2013-05-30 00:00:00.000,NULL,159.00
AWC Logo Cap,2013-05-30 00:00:00.000,NULL,8.99
AWC Logo Cap,2012-05-30 00:00:00.000,2013-05-29 00:00:00.000,8.6442
AWC Logo Cap,2011-05-31 00:00:00.000,2012-05-29 00:00:00.000,8.6442
Bike Wash - Dissolver,2013-05-30 00:00:00.000,NULL,7.95
Cable Lock,2012-05-30 00:00:00.000,2013-05-29 00:00:00.000,25.00
Chain,2013-05-30 00:00:00.000,NULL,20.24
"Classic Vest, L",2013-05-30 00:00:00.000,NULL,63.50
"Classic Vest, M",2013-05-30 00:00:00.000,NULL,63.50
"Classic Vest, S",2013-05-30 00:00:00.000,NULL,63.50


## **Complex Queries:**

### **Query 1**

**Proposition:** <span style="color: var(--vscode-foreground);">Identify the most frequently occuring Pokemon for both the primary type and secondary type categories accross the three Pokemon generations and ranks the counts based on it and select the type with the highest occurance in each category.</span>

**Tables:**

- PokemonGen1.dbo.PokemonGen1
- PokemonGen2.dbo.PokemonGen2
- PokemonGen3.dbo.PokemonGen3

**Columns:**

- Type1: The primary type of a Pokemon.
- Type2: The secondary type of a Pokemon, which can be NULL if the Pokemon does not possess a secondary type.

**Predicate:**

- The query first aggregates the counts of each Pokemon for both Type1 and Type2 across all three generations in two separate CTE tables, Type1Counts and Type2Counts. It counts the number of occurances in the tables, excluding the null values. 
- Then it ranks the types based on the numbner of occurances using the RANK() function with two CTEs, RankType1 and RankType2
- The query then selects the most common types from RankedType1 and RankedType2 where the rank is 1 by using a SELECT statement with a UNION ALL
- If there are duplicates, or a case where the count is the same, all of them are diplayed

In [18]:
WITH Type1Counts AS (
    SELECT Type1 AS Type, COUNT(*) AS Count
    FROM PokemonGen1.dbo.PokemonGen1
    WHERE Type1 IS NOT NULL
    GROUP BY Type1
    UNION ALL
    SELECT Type1, COUNT(*)
    FROM PokemonGen2.dbo.PokemonGen2
    WHERE Type1 IS NOT NULL
    GROUP BY Type1
    UNION ALL
    SELECT Type1, COUNT(*)
    FROM PokemonGen3.dbo.PokemonGen3
    WHERE Type1 IS NOT NULL
    GROUP BY Type1
),
Type2Counts AS (
    SELECT Type2 AS Type, COUNT(*) AS Count
    FROM PokemonGen1.dbo.PokemonGen1
    WHERE Type2 IS NOT NULL
    GROUP BY Type2
    UNION ALL
    SELECT Type2, COUNT(*)
    FROM PokemonGen2.dbo.PokemonGen2
    WHERE Type2 IS NOT NULL
    GROUP BY Type2
    UNION ALL
    SELECT Type2, COUNT(*)
    FROM PokemonGen3.dbo.PokemonGen3
    WHERE Type2 IS NOT NULL
    GROUP BY Type2
),
RankedType1 AS (
    SELECT Type, Count, RANK() OVER (ORDER BY Count DESC) AS Rank
    FROM Type1Counts
),
RankedType2 AS (
    SELECT Type, Count, RANK() OVER (ORDER BY Count DESC) AS Rank
    FROM Type2Counts
)

SELECT 'Type1' AS Category, Type, Count
FROM RankedType1
WHERE Rank = 1
UNION ALL
SELECT 'Type2' AS Category, Type, Count
FROM RankedType2
WHERE Rank = 1;


(4 rows affected)

Total execution time: 00:00:00.003

Category,Type,Count
Type1,Water,28
Type2,Flying,19
Type2,Poison,19
Type2,Flying,19


### **Query 2**

**Proposition:** Write a query that identifies the Pokemon types that were first introduced in Generation 2 and continue to appear in Generation 3 that illustrates the evolution and retention of Pokemon types across these generations.

**Tables:**

- PokemonGen1.dbo.PokemonGen1
- PokemonGen2.dbo.PokemonGen2
- PokemonGen3.dbo.PokemonGen3

**Columns:**

- Type1: The primary type of Pokémon.
- Type2: The secondary type of Pokémon, which can be NULL if the Pokemon does not have a secondary type.

**Predicate:**

- It uses CTEs named Gen1Types, Gen2Types, and Gen3Types and gets a distinct list of both Type1 and Type2, exlcuding null
- The CTE then fillters the types from Gen2Types by exclluding the ones in Gen1Types
- The CTE continues the filter the types from Gen3Types by excluding the ones in Gen2Types
- FInally the SELECT statement is used to list all the types that are introduced in Gen2 and are continued in Gen3. The output shows the types that were introduced in the middle generation and then adopted in the next generation.

In [21]:
WITH Gen1Types AS (
    SELECT DISTINCT Type1 AS Type FROM PokemonGen1.dbo.PokemonGen1
    UNION
    SELECT DISTINCT Type2 FROM PokemonGen1.dbo.PokemonGen1 WHERE Type2 IS NOT NULL
), 
Gen2Types AS (
    SELECT DISTINCT Type1 AS Type FROM PokemonGen2.dbo.PokemonGen2
    UNION
    SELECT DISTINCT Type2 FROM PokemonGen2.dbo.PokemonGen2 WHERE Type2 IS NOT NULL
),
Gen3Types AS (
    SELECT DISTINCT Type1 AS Type FROM PokemonGen3.dbo.PokemonGen3
    UNION
    SELECT DISTINCT Type2 FROM PokemonGen3.dbo.PokemonGen3 WHERE Type2 IS NOT NULL
),
IntroducedInGen2 AS (
    SELECT Type FROM Gen2Types
    WHERE Type NOT IN (SELECT Type FROM Gen1Types)
),
ContinuedInGen3 AS (
    SELECT Type FROM IntroducedInGen2
    WHERE Type IN (SELECT Type FROM Gen3Types)
)

SELECT Type
FROM ContinuedInGen3;


(1 row affected)

Total execution time: 00:00:00.013

Type
Dark


### **Query 3**

**Proposition:** Write a query to find the Pokemon that has the highest attack in Type1 and Type2 within each generation

**Tables:**

- PokemonGen1.dbo.PokemonGen1
- PokemonGen2.dbo.PokemonGen2
- PokemonGen3.dbo.PokemonGen3

**Columns:**

- ID (or PokemonNoGen1, PokemonNoGen2, PokemonNoGen3) 
- Name 
- Attack
- Type1 and Type2: Represent the primary and secondary types of the Pokemon.
- Generation 

**Predicate:**

- The UnifiedPokemon CTE is used to create a unified view of the Pokemon from all 3 generations by getting the IDs, names, attack values, types, and assigns a generation tag based on the source table
- Then the TypeRankings CTE creates a temporary dataset that consolidates the Pokemon types and then ranks the Pokemon by their attack values within each type and generation using the RANK() function
- The query then selects the TypeRankings CTE and filters for when the Pokemon with the highest attack within the type and generation

In [30]:
WITH UnifiedPokemon AS (
    SELECT PokemonNoGen1 AS ID, Name, Attack, Type1, Type2, 'Gen1' AS Generation FROM PokemonGen1.dbo.PokemonGen1
    UNION ALL
    SELECT PokemonNoGen2 AS ID, Name, Attack, Type1, Type2, 'Gen2' FROM PokemonGen2.dbo.PokemonGen2
    UNION ALL
    SELECT PokemonNoGen3 AS ID, Name, Attack, Type1, Type2, 'Gen3' FROM PokemonGen3.dbo.PokemonGen3
),
TypeRankings AS (
    SELECT
        Name,
        Attack,
        Type,
        Generation,
        RANK() OVER(PARTITION BY Type, Generation ORDER BY Attack DESC) AS RankInType
    FROM (
        SELECT Name, Attack, Type1 AS Type, Generation FROM UnifiedPokemon
        UNION ALL
        SELECT Name, Attack, Type2, Generation FROM UnifiedPokemon WHERE Type2 IS NOT NULL
    ) AS AllTypes
)

SELECT Type, Generation, Name, Attack
FROM TypeRankings
WHERE RankInType = 1
ORDER BY Type, Generation;


(62 rows affected)

Total execution time: 00:00:00.010

Type,Generation,Name,Attack
Bug,Gen1,Pinsir,125
Bug,Gen2,Scizor,130
Bug,Gen3,Armaldo,125
Dark,Gen2,Tyranitar,134
Dark,Gen3,Absol,130
Dragon,Gen1,Dragonite,134
Dragon,Gen2,Kingdra,95
Dragon,Gen3,Rayquaza,150
Electric,Gen1,Zapdos,90
Electric,Gen1,Raichu,90


### **Query 4**

**Proposition:** Create a query that finds the most common abilities for Ability1, Ability2, and Ability3 from the 3 Pokemon generations

**Tables:**

- PokemonGen1.dbo.PokemonGen1 
- PokemonGen2.dbo.PokemonGen2 
- PokemonGen3.dbo.PokemonGen3 

**Columns:**

- AbilitySlot (Ability1, Ability2, or Ability3)
- Ability 
- Count 

**Predicate:**

- Using the Abilities CTE, take all the abilites from the Pokemon from 3 generations and assign each ability to a slot in the table unless it is null
- Use the AbilityCounts CTE to group the AbilitySlot and Ability that keeps track of the count in each slot
- Then the MaxCounts CTE is used to count the abilites in each slot and determines the max count in each group
- If there are duplicates, they are still shown

In [34]:
WITH Abilities AS (
    SELECT 'Ability1' AS AbilitySlot, Ability1 AS Ability FROM PokemonGen1.dbo.PokemonGen1
    UNION ALL
    SELECT 'Ability2' AS AbilitySlot, Ability2 FROM PokemonGen1.dbo.PokemonGen1 WHERE Ability2 IS NOT NULL
    UNION ALL
    SELECT 'Ability3' AS AbilitySlot, Ability3 FROM PokemonGen1.dbo.PokemonGen1 WHERE Ability3 IS NOT NULL
    UNION ALL
    SELECT 'Ability1' AS AbilitySlot, Ability1 FROM PokemonGen2.dbo.PokemonGen2
    UNION ALL
    SELECT 'Ability2' AS AbilitySlot, Ability2 FROM PokemonGen2.dbo.PokemonGen2 WHERE Ability2 IS NOT NULL
    UNION ALL
    SELECT 'Ability3' AS AbilitySlot, Ability3 FROM PokemonGen2.dbo.PokemonGen2 WHERE Ability3 IS NOT NULL
    UNION ALL
    SELECT 'Ability1' AS AbilitySlot, Ability1 FROM PokemonGen3.dbo.PokemonGen3
    UNION ALL
    SELECT 'Ability2' AS AbilitySlot, Ability2 FROM PokemonGen3.dbo.PokemonGen3 WHERE Ability2 IS NOT NULL
    UNION ALL
    SELECT 'Ability3' AS AbilitySlot, Ability3 FROM PokemonGen3.dbo.PokemonGen3 WHERE Ability3 IS NOT NULL
),
AbilityCounts AS (
    SELECT AbilitySlot, Ability, COUNT(*) AS Count
    FROM Abilities
    GROUP BY AbilitySlot, Ability
),
MaxCounts AS (
    SELECT AbilitySlot, MAX(Count) AS MaxCount
    FROM AbilityCounts
    GROUP BY AbilitySlot
)

SELECT ac.AbilitySlot, ac.Ability, ac.Count
FROM AbilityCounts ac
JOIN MaxCounts mc ON ac.AbilitySlot = mc.AbilitySlot AND ac.Count = mc.MaxCount
ORDER BY ac.AbilitySlot, ac.Count DESC;


(6 rows affected)

Total execution time: 00:00:00.005

AbilitySlot,Ability,Count
Ability1,Chlorophyll,19
Ability1,Swift Swim,19
Ability2,Early Bird,9
Ability2,Flash Fire,9
Ability2,Sturdy,9
Ability3,Sheer Force,10


### **Query 5**

**Proposition:** Write a query that retrieves the Pokemon from all 3 generations that have a Defense value higher than the Special Defense Value

**Tables:**

- PokemonGen1.dbo.PokemonGen1 
- PokemonGen2.dbo.PokemonGen2 
- PokemonGen3.dbo.PokemonGen3 

**Columns:**

- Generation 
- Name 
- Defense 
- SpecialDefense 

**Predicate:**

- There are 3 subqueries for each generation of Pokemon, it takes the generation name and then retrieves the name of the Pokemon, Defense, and SpecialDefense stats from the Pokemon table, then the data is filtered to only include the Pokemon that have a Defense higher than the Special Defense
- The UNION ALL operator is then used to combine the results from all three of the subqueries
- The outer query then takes the columns from the combined results and adds a Generation column to specify the generation of the Pokemon

In [35]:
SELECT *
FROM (
    SELECT 'Generation 1' AS Generation, Name, Defense, SpecialDefense
    FROM PokemonGen1.dbo.PokemonGen1
    WHERE Defense > SpecialDefense

    UNION ALL

    SELECT 'Generation 2' AS Generation, Name, Defense, SpecialDefense
    FROM PokemonGen2.dbo.PokemonGen2
    WHERE Defense > SpecialDefense

    UNION ALL

    SELECT 'Generation 3' AS Generation, Name, Defense, SpecialDefense
    FROM PokemonGen3.dbo.PokemonGen3
    WHERE Defense > SpecialDefense
) AS AllGenerations
ORDER BY Generation, Name;


(123 rows affected)

Total execution time: 00:00:00.008

Generation,Name,Defense,SpecialDefense
Generation 1,Rhydon,120,45
Generation 1,Tangela,115,40
Generation 1,Bellsprout,35,30
Generation 1,Caterpie,35,20
Generation 1,Cloyster,180,45
Generation 1,Cubone,95,50
Generation 1,Dodrio,70,60
Generation 1,Doduo,45,35
Generation 1,Exeggcute,80,45
Generation 1,Exeggutor,85,75


### **Query 6**

**Proposition:** Write a query that calculates the average values of the attributes of Pokemon: HP, Attack, Defense, Special Attack, Special Defense, and Speed across all 3 generations

**Tables:**

- PokemonGen1.dbo.PokemonGen1 
- PokemonGen2.dbo.PokemonGen2 
- PokemonGen3.dbo.PokemonGen3 

**Columns:**

- HP 
- Attack 
- Defense 
- SpecialAttack 
- SpecialDefense
- Speed 

**Predicate:**

- The UNION ALL is used with a subquery that takes the data from all 3 generations of Pokemon to combine the  HP, Attack, Defense, Special Attack, Special Defense, and Speed
- The outer query then calculates the average of each attribute using the AVG() function

In [38]:
SELECT 
    AVG(HP) AS Average_HP,
    AVG(Attack) AS Average_Attack,
    AVG(Defense) AS Average_Defense,
    AVG(SpecialAttack) AS Average_SpecialAttack,
    AVG(SpecialDefense) AS Average_SpecialDefense,
    AVG(Speed) AS Average_Speed
FROM (
    SELECT HP, Attack, Defense, SpecialAttack, SpecialDefense, Speed
    FROM PokemonGen1.dbo.PokemonGen1

    UNION ALL

    SELECT HP, Attack, Defense, SpecialAttack, SpecialDefense, Speed
    FROM PokemonGen2.dbo.PokemonGen2

    UNION ALL

    SELECT HP, Attack, Defense, SpecialAttack, SpecialDefense, Speed
    FROM PokemonGen3.dbo.PokemonGen3
) AS AllGenerations;

(1 row affected)

Total execution time: 00:00:00.003

Average_HP,Average_Attack,Average_Defense,Average_SpecialAttack,Average_SpecialDefense,Average_Speed
66,71,68,66,67,64


### **Query 7**

**Proposition:** Write a query that compares the average stats for HP, Attack, Defense, Special Attack, Special Defense, and Speed for all 3 generations of Pokemon

**Tables:**

- PokemonGen1.dbo.PokemonGen1 
- PokemonGen2.dbo.PokemonGen2 
- PokemonGen3.dbo.PokemonGen3 

**Columns:**

- Generation 
- AvgHP 
- AvgAttack 
- AvgDefense 
- AvgSpecialAttack 
- AvgSpecialDefense 
- AvgSpeed 

**Predicate:**

- The ConbinedStats CTE is used to calculate the average for all the attributes that the Pokemon have
- Then there are 3 subqueries that are used for each generation to calculate the average
- The UNION ALL operator is then used to combine the results of the subqueries in to the CombinedStats table

In [40]:
WITH CombinedStats AS (
    SELECT 'Gen1' AS Generation, 
           AVG(HP) AS AvgHP, 
           AVG(Attack) AS AvgAttack, 
           AVG(Defense) AS AvgDefense,
           AVG(SpecialAttack) AS AvgSpecialAttack,
           AVG(SpecialDefense) AS AvgSpecialDefense,
           AVG(Speed) AS AvgSpeed
    FROM PokemonGen1.dbo.PokemonGen1
    UNION ALL
    SELECT 'Gen2', 
           AVG(HP), 
           AVG(Attack), 
           AVG(Defense),
           AVG(SpecialAttack),
           AVG(SpecialDefense),
           AVG(Speed)
    FROM PokemonGen2.dbo.PokemonGen2
    UNION ALL
    SELECT 'Gen3', 
           AVG(HP), 
           AVG(Attack), 
           AVG(Defense),
           AVG(SpecialAttack),
           AVG(SpecialDefense),
           AVG(Speed)
    FROM PokemonGen3.dbo.PokemonGen3
)
SELECT Generation, 
       AvgHP, 
       AvgAttack, 
       AvgDefense,
       AvgSpecialAttack,
       AvgSpecialDefense,
       AvgSpeed
FROM CombinedStats;


(3 rows affected)

Total execution time: 00:00:00.006

Generation,AvgHP,AvgAttack,AvgDefense,AvgSpecialAttack,AvgSpecialDefense,AvgSpeed
Gen1,63,72,68,67,65,69
Gen2,70,68,69,64,72,61
Gen3,65,73,69,67,66,61
